### PROJETO 1: Raspagem de dados com Selenium e carregando em um Banco de Dados com SQL

#### Neste projeto será utilizado o Site "Books to Scrape" que é um site fictício projetado especificamente para prática de raspagem de dados. Ele contém várias categorias de livros, com informaçöes como título, preço, etc...

* Site: http://books.toscrape.com/index.html

In [388]:
# importando as biblióteca 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
drive = webdriver.Chrome(service=servico)


In [389]:
site = "http://books.toscrape.com/index.html"
drive.get(site)

In [390]:
# Criando listas para armazenamento das informaçöes
nome = []
preco = []
status = []

In [391]:
#Pegando a quantidade de paginas do site
resultado= int(drive.find_element(By.XPATH,'//*[@id="default"]/div/div/div/div/form/strong[1]').text)
mostrador = int(drive.find_element(By.XPATH,'//*[@id="default"]/div/div/div/div/form/strong[3]').text)
num_paginas = int(resultado/mostrador)

In [392]:
# Realizando a raspagem dos dados selecionados
import time
i= 0 
try:
    while i <= num_paginas: 
        tabela = drive.find_element(By.XPATH,'//*[@id="default"]/div/div/div/div/section/div[2]/ol')
        livros = tabela.find_elements(By.CLASS_NAME, 'product_pod')
        for livro  in livros:
            h3 = livro.find_element(By.TAG_NAME,'h3')
            nome.append(h3.find_element(By.TAG_NAME,'a').get_attribute('title'))
            preco.append(livro.find_element(By.CLASS_NAME,'price_color').text)
            status.append(livro.find_element(By.CLASS_NAME,'product_price').text[7:14])
        page = drive.find_element(By.CLASS_NAME, 'pager')
        page = page.find_element(By.CLASS_NAME,'next')
        next = page.find_element(By.TAG_NAME, 'a').click()
        time.sleep(1)
        i += 1
except:
    pass


In [393]:
# Criando um DataFrame Pandas com os dados 
df = pd.DataFrame(columns=['Nome','Status','Preço'])
df['Nome'] = nome
df['Preço'] = preco
df['Status'] = status
df[['Moeda','Preço']]= df['Preço'].str.split(pat='£',regex=None,expand=True)
df['Moeda'] = '£'
display(df.shape) # Número de linhas / colunas
display(df.head()) # Imprime as 5 primeiras linhas
display(df.tail()) # Imprime as 5 ultimas linhas 

(80, 4)

,Nome,Status,Preço,Moeda
0,A Light in the Attic,In stoc,51.77,£
1,Tipping the Velvet,In stoc,53.74,£
2,Soumission,In stoc,50.10,£
3,Sharp Objects,In stoc,47.82,£
4,Sapiens: A Brief History of Humankind,In stoc,54.23,£


,Nome,Status,Preço,Moeda
75,Security,In stoc,39.25,£
76,"Saga, Volume 6 (Saga (Collected Editions) #6)",In stoc,25.02,£
77,"Saga, Volume 5 (Saga (Collected Editions) #5)",In stoc,51.04,£
78,Reskilling America: Learning to Labor in the T...,In stoc,19.83,£
79,"Rat Queens, Vol. 3: Demons (Rat Queens (Collec...",In stoc,50.40,£


## Realizando a integraçao com Banco de Dados SQL SERVER
* Create (Criar) , Read (Ler), Update(atualizar) , Delete(apagar) 

In [395]:
import pyodbc

In [396]:
def conect_ao_banco(driver='SQL Server', server='DESKTOP-82IHFV4', data='Raspagem_Dados',username=None, passorwd=None,truested_connection='yes'):\

    string_conexao = f"Driver={driver};Server={server};Database={data};UID={username};PWD={passorwd}; TRUSTED_CONNECTION={truested_connection}"

    conexao = pyodbc.connect(string_conexao)
    cursor = conexao.cursor()
    
    return conexao,cursor

conexao, cursor = conect_ao_banco()
print("Conexão bem sucedida!")

Conexão bem sucedida!


In [402]:
for linha in df.index:
    
    nome = (df.loc[linha,'Nome']).replace("'",'')
    status = df.loc[linha,'Status']
    preco = df.loc[linha,'Preço']
    moeda= df.loc[linha,'Moeda']
    cursor.execute(f"INSERT INTO Raspagem(Nome,Status,Preco,Moeda) VALUES ('{nome}','{status}',{preco},'{moeda}')")
    cursor.commit()
    
    

In [ ]:
conexao.close()